# Week 3: Preprocessing and Walkthrough

Aniruddh K B and Rachit Yagnik

## Acknowledgements: 

This content borrows heavily from last year's material prepared by the TAs. The TAs at that time were:

Vibhav Agarwal (IMT2016),
Tejas Kotha (IMT2016),
Tanmay Jain (IMT2016),
Shreyas Gupta (IMT2016),
Saurabh Jain (IMT2016),
Divyanshu Khandelwal (IMT2016),
Arjun Verma (IMT2017),
Amitesh Anand (MT2019),
Mohd Zahid Faiz (MT2019),
Tushar Anil Masane (MT2019)


# 1. Preprocessing

So far, we have been considering data to be in the form of neatly packaged matrices of floating-point numbers. As you might imagine, real life is never quite so simple. 

There are numerous issues which might crop up with data, but we will be addressing only a few of them here. The aim is to give you a little bit of information that will be helpful for the first assignment.

**Important note: When working with tabular data, ALWAYS, ALWAYS merge your training and test data, perform the preprocessing steps on this combined data, and then split the data. NEVER preprocess them separately!**

## 1.1. Imports and introduction

In [359]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder

main_df = pd.read_csv("datasets/preprocessing/to_work_on.csv")

Let's have a look at the data.

In [360]:
main_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,Heating,CentralAir,Electrical,TotRmsAbvGrd,FireplaceQu,ScreenPorch,YrSold,SalePrice
0,1,60,RL,65.0,8450,NaN,Reg,706,150,856,GasA,Y,SBrkr,8,NaN,0,2008,208500
1,2,20,RL,80.0,9600,NaN,Reg,978,284,1262,GasA,Y,SBrkr,6,TA,0,2007,181500
2,3,60,RL,68.0,11250,NaN,IR1,486,434,920,GasA,Y,SBrkr,6,TA,0,2008,223500
3,4,70,RL,60.0,9550,NaN,IR1,216,540,756,GasA,Y,SBrkr,7,Gd,0,2006,140000
4,5,60,RL,84.0,14260,NaN,IR1,655,490,1145,GasA,Y,SBrkr,9,TA,0,2008,250000


In [361]:
main_df.describe()

,Id,MSSubClass,LotFrontage,LotArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,TotRmsAbvGrd,ScreenPorch,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,443.639726,567.240411,1057.429452,6.517808,15.060959,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,456.098091,441.866955,438.705324,1.625393,55.757415,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,0.000000,223.000000,795.750000,5.000000,0.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,383.500000,477.500000,991.500000,6.000000,0.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,712.250000,808.000000,1298.250000,7.000000,0.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,5644.000000,2336.000000,6110.000000,14.000000,480.000000,2010.000000,755000.000000


Two important things should jump out at you: 

1. We have missing values. 
2. The data isn't only numeric.


## 1.2. Dealing with missing values 

1. Drop the rows with null values.

This can be done if you have plenty of good data available and the loss of a few rows doesn't impact the training much.

2. Drop the columns with null values.

This should be done if there is a lot of data missing from a particular column and no suitable default value can be inferred for the missing values.

3. Replace the null values with the mean, median, mode or a default value. 

This should be done if there is not much data missing in a column, the column is important for training, OR if a sensible default value exists.

4. Use sophisticated methods to infer the data (clustering or supervised learning).

We won't be covering this here, but the main idea is to think of the data with missing values as a target column (y) and to use the remaining features (X) to somehow predict the missing values. This method is accurate, but difficult. It also needs at least _some_ useful data to be present for the missing column, and for the missing column to somehow be related to the other columns.

**Much of machine learning is more art than science** -- including this step here. If you are doubtful of the effect of an individual choice, and computation/time is not a concern, you can always try both approaches and see what happens.

Now, we will demonstrate the first three approaches.

Note: In some datasets, a missing value may be denoted with "?". Check those out too, using df[column].unique()

In [362]:
main_df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Alley           1369
LotShape           0
BsmtFinSF1         0
BsmtUnfSF          0
TotalBsmtSF        0
Heating            0
CentralAir         0
Electrical         1
TotRmsAbvGrd       0
FireplaceQu      690
ScreenPorch        0
YrSold             0
SalePrice          0
dtype: int64

### 1.2.1 Deleting rows

"Electrical" has just one missing value. Since we have enough data, removing 1 row probably won't hurt the process. Let's do that. 

In [363]:
main_df.loc[main_df["Electrical"].isna(), :] #Displays the row which is null in "Electrical".

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,Heating,CentralAir,Electrical,TotRmsAbvGrd,FireplaceQu,ScreenPorch,YrSold,SalePrice
1379,1380,80,RL,73.0,9735,NaN,Reg,0,384,384,GasA,Y,NaN,7,NaN,0,2008,167500


In [364]:
main_df.index[main_df["Electrical"].isna()] 
# Returns all indices with "na" in "Electrical" column

Int64Index([1379], dtype='int64')

In [365]:
main_df.drop(axis="rows", labels=main_df.index[main_df["Electrical"].isna()], inplace=True)

In [366]:
main_df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Alley           1368
LotShape           0
BsmtFinSF1         0
BsmtUnfSF          0
TotalBsmtSF        0
Heating            0
CentralAir         0
Electrical         0
TotRmsAbvGrd       0
FireplaceQu      689
ScreenPorch        0
YrSold             0
SalePrice          0
dtype: int64

The row with one missing value in Electrical is gone.

### 1.2.2. Dropping problematic columns

In [367]:
main_df["Alley"].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

Here, we have two choices: We replace with a default value (say, "noAlley"), or we drop the column entirely. The choice is yours. For demonstration's sake, I will drop this column.

In [368]:
main_df.drop(axis="columns", labels="Alley", inplace=True) 
#Think about why we're using inplace=True everywhere.

In [369]:
main_df.isna().sum()

Id                0
MSSubClass        0
MSZoning          0
LotFrontage     259
LotArea           0
LotShape          0
BsmtFinSF1        0
BsmtUnfSF         0
TotalBsmtSF       0
Heating           0
CentralAir        0
Electrical        0
TotRmsAbvGrd      0
FireplaceQu     689
ScreenPorch       0
YrSold            0
SalePrice         0
dtype: int64

### 1.2.3. Replacing with mean/median/mode/default value

In [370]:
main_df["FireplaceQu"].unique()

array([nan, 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [371]:
main_df["FireplaceQu"].value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

Here's where domain knowledge comes in. Maybe a NA means there is no fireplace? We could fill with a new category "noFireplace".

In [372]:
main_df["FireplaceQu"].fillna("noFireplace", inplace=True)

In [373]:
main_df["FireplaceQu"].value_counts()

noFireplace    689
Gd             380
TA             313
Fa              33
Ex              24
Po              20
Name: FireplaceQu, dtype: int64

In [374]:
main_df.isna().sum()

Id                0
MSSubClass        0
MSZoning          0
LotFrontage     259
LotArea           0
LotShape          0
BsmtFinSF1        0
BsmtUnfSF         0
TotalBsmtSF       0
Heating           0
CentralAir        0
Electrical        0
TotRmsAbvGrd      0
FireplaceQu       0
ScreenPorch       0
YrSold            0
SalePrice         0
dtype: int64

In [375]:
main_df["LotFrontage"].value_counts()

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64

In [376]:
main_df["LotFrontage"].describe()

count    1200.000000
mean       70.047500
std        24.294727
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

This is a continuous variable. The mean and the median (50 %ile) are pretty close, and the mode is not very far from there either. We could use any of the three to fill this column. 

If there are a small or moderate number of NANs and the data is categorical in nature, the mode is a nice value to fill.

In [377]:
main_df["LotFrontage"].fillna(main_df["LotFrontage"].median(), inplace=True)

In [378]:
main_df.isna().sum()

Id              0
MSSubClass      0
MSZoning        0
LotFrontage     0
LotArea         0
LotShape        0
BsmtFinSF1      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
CentralAir      0
Electrical      0
TotRmsAbvGrd    0
FireplaceQu     0
ScreenPorch     0
YrSold          0
SalePrice       0
dtype: int64

## 1.3. Dealing with categorical and non-numeric data

In most cases (where the column is useful for our purposes) non-numeric data is *categorical* in nature. What this means is that the column has a small number of unique values or **categories**. It could be something like a yes/no question, or more complex than that. 

Suppose we have a column "colour" where there are three categories -- red, blue and green. 

One thing we could do is assign "red" -> 0, "blue" -> 1, "green" -> 2 and so on. This is called **label encoding**.

Another option is to have a dedicated column each for red, blue and green. When the original column is "red", make the entry for the red column 1 and the other two columns zero. And similar for blue and green. This is called **one-hot encoding**. 

### 1.3.1. Label encoding

In [379]:
column_to_label_encode = main_df["MSZoning"]
column_to_label_encode.head()

0    RL
1    RL
2    RL
3    RL
4    RL
Name: MSZoning, dtype: object

In [380]:
label_encoder = LabelEncoder()
label_encoded_column = label_encoder.fit_transform(column_to_label_encode)
label_encoded_column

array([3, 3, 3, ..., 3, 3, 3])

What we did here: 

1. Take the column you wish to label encode. 
2. Make a label encoder.
3. Pass the column to the label encoder. 

Note: It would be better to create separate label encoders for different columns if you wish to use the label encoder on test data later on as well (when reusing on test data, use the transform method, not fit_transform).

Note2: You can only pass an individual column to the SKLearn Label Encoder.

Now, you can replace the column in the original data. 

However, label encoding has many issues -- even though it is very space-efficient.

1. It creates an implicit ordering within the data which is sometimes undesirable. For instance, if red -> 0, blue-> 1, green -> 3 , does that mean that red < blue < green ? No. 
2. What about the values in between the labels? For instance, what of the value 1.5 ? That value has no meaning in our encoding -- but the model doesn't know that! 

It is for these reasons that we usually prefer one-hot encoding*, UNLESS we have too many columns or too many clases within a particular column. 


*Footnote (can ignore for now): Tree based methods like decision trees, random forests, and gradient boosted trees work well with label encoding, and in some cases, better than with one-hot encoding.

### 1.3.2. One-hot encoding:

In [381]:
column_names_to_one_hot = ["LotShape", "Heating"]

In [382]:
main_df = pd.get_dummies(main_df, columns=column_names_to_one_hot)

In [383]:
main_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,CentralAir,Electrical,...,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
0,1,60,RL,65.0,8450,706,150,856,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
1,2,20,RL,80.0,9600,978,284,1262,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
2,3,60,RL,68.0,11250,486,434,920,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
3,4,70,RL,60.0,9550,216,540,756,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
4,5,60,RL,84.0,14260,655,490,1145,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0


In [384]:
main_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', 'CentralAir', 'Electrical', 'TotRmsAbvGrd',
       'FireplaceQu', 'ScreenPorch', 'YrSold', 'SalePrice', 'LotShape_IR1',
       'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'Heating_Floor',
       'Heating_GasA', 'Heating_GasW', 'Heating_Grav', 'Heating_OthW',
       'Heating_Wall'],
      dtype='object')

## 1.4. Duplicate rows:

In [385]:
main_df.index[main_df.duplicated()]

Int64Index([], dtype='int64')

There are no duplicate rows here. Though, strictly speaking, we should do this after removing any serial-number columns.


Let's say, for the sake of the argument, that we introduce some duplicate columns.

In [386]:
main_df = main_df.append(main_df.iloc[:5,:]) 
# We're copying the first five rows again to the end.

In [387]:
main_df.duplicated().sum()

5

In [388]:
main_df.drop(axis="rows", labels=main_df.index[main_df.duplicated()], inplace=True)

In [389]:
main_df.duplicated().sum()

0

## 1.5. Train-test split
The reasons for train-test split should be discussed by the TA. TLDR: To prevent overfitting, you train using a training dataset and compare against a test dataset which the model has not seen during training. 

In [390]:
from sklearn.model_selection import train_test_split 

train_df, test_df = train_test_split(main_df, train_size=0.9)

In [391]:
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,CentralAir,Electrical,...,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
181,182,70,RL,54.0,7588,352,441,793,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
753,754,60,RL,80.0,10240,0,1030,1030,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
1166,1167,20,RL,64.0,10475,0,1694,1694,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
856,857,80,RL,69.0,10970,505,0,940,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
95,96,60,RL,69.0,9765,310,370,680,Y,SBrkr,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,554,20,RL,67.0,8777,0,0,0,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
252,253,60,RL,65.0,8366,0,798,798,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
292,293,50,RL,60.0,11409,292,476,768,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
565,566,70,RL,66.0,6858,0,806,806,N,FuseF,...,0,0,0,1,0,1,0,0,0,0


In [392]:
test_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,CentralAir,Electrical,...,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
832,833,60,RL,44.0,9548,483,458,941,Y,SBrkr,...,1,0,0,0,0,1,0,0,0,0
185,186,75,RM,90.0,22950,0,1107,1107,Y,SBrkr,...,0,1,0,0,0,1,0,0,0,0
542,543,20,RL,78.0,10140,144,379,1650,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
361,362,50,RL,69.0,9144,399,484,883,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
1062,1063,190,RM,85.0,13600,0,662,662,N,SBrkr,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,1425,20,RL,69.0,9503,457,193,1024,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
788,789,20,RM,50.0,6000,0,901,901,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
373,374,20,RL,79.0,10634,428,0,608,Y,SBrkr,...,0,0,0,1,0,1,0,0,0,0
104,105,50,RM,69.0,7758,224,816,1040,Y,FuseF,...,0,0,0,1,0,1,0,0,0,0


# 2. An example linear regression problem, from beginning to end

### 2.1. Imports and intro 

Just reimporting stuff, so that it's clear what all we need: 

In [393]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [394]:
main_df = pd.read_csv("./datasets/cardekho/car_data.csv")

In [395]:
main_df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


The main objective is to predict the Selling_Price , given the rest of the data available to us. A detailed description of the data is available at https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho 

### 2.2. Checking for missing values

In [396]:
main_df.isna().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

No missing values. But wait...

In [397]:
(main_df == "?").sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

Okay, I guess there are no missing values. 

### 2.3. Categorical --> Numeric

First, we must evaluate the columns and see if one-hot encodings are viable.

In [398]:
main_df.nunique()

Car_Name          98
Year              16
Selling_Price    156
Present_Price    147
Kms_Driven       206
Fuel_Type          3
Seller_Type        2
Transmission       2
Owner              3
dtype: int64

Car_Name, Fuel_Type, Seller_Type and Transmission can be thought of as categorical. The problem is, Car_Name has way too many unique values to be useful. Since we are restricted to Naive Bayes/Multivariate Bayes/Logistic Regression, it doesn't make sense to use this column.

In [399]:
main_df.drop(axis="columns", labels="Car_Name", inplace=True)

In [400]:
main_df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [401]:
column_names_to_one_hot = ["Fuel_Type", "Seller_Type", "Transmission"]

In [402]:
main_df = pd.get_dummies(main_df, columns=column_names_to_one_hot)

In [403]:
main_df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,2014,3.35,5.59,27000,0,0,0,1,1,0,0,1
1,2013,4.75,9.54,43000,0,0,1,0,1,0,0,1
2,2017,7.25,9.85,6900,0,0,0,1,1,0,0,1
3,2011,2.85,4.15,5200,0,0,0,1,1,0,0,1
4,2014,4.60,6.87,42450,0,0,1,0,1,0,0,1


In [404]:
main_df.columns

Index(['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Owner',
       'Fuel_Type_CNG', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol',
       'Seller_Type_Dealer', 'Seller_Type_Individual',
       'Transmission_Automatic', 'Transmission_Manual'],
      dtype='object')

### 2.4. Duplicate rows

In [405]:
main_df.duplicated().sum()

2

In [406]:
main_df.drop(axis="rows", labels=main_df.index[main_df.duplicated()], inplace=True)

In [407]:
main_df.duplicated().sum()

0

### 2.5. Training and train-test-split

In [408]:
X = main_df.drop(axis="columns", labels="Selling_Price").to_numpy().astype(np.float64)
y = main_df["Selling_Price"].to_numpy().astype(np.float64)

In [409]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.6)

If the above step fails, it means that there is some data that is still non-numeric. This could be because we haven't dealt with all missing data, some column that we thought is numeric is "object" for some reason, or that we have forgotten to encode a column.

Ignore for now: This is also the point at which we would do a train-dev-test split of the data.

In [410]:
lr_model = LinearRegression()
lr_model.fit(train_X, train_y)

LinearRegression()

In [411]:
train_yhat = lr_model.predict(train_X)

In [412]:
mean_squared_error(train_y, train_yhat)

2.283608447019485

In [413]:
mean_absolute_error(train_y, train_yhat)

0.9959273251552203

In [414]:
test_yhat = lr_model.predict(test_X)

In [415]:
mean_squared_error(test_y, test_yhat)

8.232006705430576

In [416]:
mean_absolute_error(test_y, test_yhat)

1.457025294051464

So, on average, the model is off by a lakh or so in the predictions. Not too bad, considering that we have dropped one of the most important columns for the regression -- the model of the car itself! For instance, Toyotas are famous for having a high resale value due to reliability and quality, while some German makes are infamous for being expensive to maintain in their later years, thus commanding a lower price in the resale market. 

Try to fiddle with the hyperparameters of the model https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html , and see if you can extract any information from the car model column -- maybe you could just extract the company, and drop the specific model? 

# 3. An example classification problem from start to end

Here, we use data taken from ECG and bloodwork of patients to predict whether they have heart disease or not. To know more about the feature columns, read the .names file in the datasets/cleveland_heart_disease folder.

Data taken from the UCI ML database.

## 3.1. Imports and introduction

In [417]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [418]:
main_df = pd.read_csv("./datasets/cleveland_heart_disease/heart_disease.data")

In [419]:
main_df.head()

,63.0,1.0,1.0.1,145.0,233.0,1.0.2,2.0,150.0,0.0,2.3,3.0,0.0.1,6.0,0
0,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
1,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
2,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
3,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
4,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


Oops! Pandas thought that the first row was the header. 

In [420]:
main_df = pd.read_csv("./datasets/cleveland_heart_disease/heart_disease.data", header=None)

In [421]:
main_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Better, but we need to rename the columns. See the .names file in the datasets/cleveland_heart_disease folder. 

In [422]:
names_mapper = {
    0:"age",
    1:"sex",
    2:"cp", 
    3:"trestbps", 
    4:"chol", 
    5:"fbs", 
    6:"restecg", 
    7:"thalach", 
    8:"exang", 
    9:"oldpeak", 
    10:"slope", 
    11:"ca", 
    12:"thal", 
    13:"target"
}

main_df.rename(axis="columns", mapper=names_mapper, inplace=True)

In [423]:
main_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [424]:
main_df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,4.000000


## 3.2. Missing values

In [425]:
main_df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [426]:
(main_df == "?").sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
target      0
dtype: int64

We have missing values in "ca" and "thal" columns. The "ca" column seems to be categorical with four categories -- but the categories have some natural ordering. 
The "thal" column is also categorical with three categories -- with no ordering in the categories.

In [427]:
main_df["ca"].describe()

count     303
unique      5
top       0.0
freq      176
Name: ca, dtype: object

In [428]:
main_df["ca"].value_counts()

0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: ca, dtype: int64

By and large, most people have ca=0, so we can make that replacement. 

In [429]:
main_df["ca"].replace("?", 0, inplace=True)
main_df["ca"] = pd.to_numeric(main_df["ca"])

In [430]:
main_df["ca"].describe()

count    303.000000
mean       0.663366
std        0.934375
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        3.000000
Name: ca, dtype: float64

In [431]:
main_df["ca"].value_counts()

0.0    180
1.0     65
2.0     38
3.0     20
Name: ca, dtype: int64

In [432]:
main_df["thal"].value_counts()

3.0    166
7.0    117
6.0     18
?        2
Name: thal, dtype: int64

Here, the majority class is 3.0, so we can safely replace the missing values with 3.0.

In [433]:
main_df["thal"].replace("?", 3.0, inplace=True)
main_df["thal"] = pd.to_numeric(main_df["thal"])

In [434]:
main_df["thal"].value_counts()

3.0    168
7.0    117
6.0     18
Name: thal, dtype: int64

In [435]:
main_df["thal"].describe()

count    303.000000
mean       4.722772
std        1.938383
min        3.000000
25%        3.000000
50%        3.000000
75%        7.000000
max        7.000000
Name: thal, dtype: float64

## 3.3 Categorical -> numeric

In [436]:
column_names_to_one_hot = [ 
    "cp",
    "restecg", 
    "slope", 
    "thal"
]

Also, we have to convert the target to 1/0 form since we won't be predicting the severity.

In [437]:
main_df["target"]  = (main_df["target"] > 0).astype("float")

In [438]:
main_df["target"]

0      0.0
1      1.0
2      1.0
3      0.0
4      0.0
      ... 
298    1.0
299    1.0
300    1.0
301    1.0
302    0.0
Name: target, Length: 303, dtype: float64

In [439]:
main_df = pd.get_dummies(main_df, columns=column_names_to_one_hot)

In [440]:
main_df.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,target,...,cp_4.0,restecg_0.0,restecg_1.0,restecg_2.0,slope_1.0,slope_2.0,slope_3.0,thal_3.0,thal_6.0,thal_7.0
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0.0,0.0,...,0,0,0,1,0,0,1,0,1,0
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,3.0,1.0,...,1,0,0,1,0,1,0,1,0,0
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,2.0,1.0,...,1,0,0,1,0,1,0,0,0,1
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0.0,...,0,1,0,0,0,0,1,1,0,0
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0.0,...,0,0,0,1,1,0,0,1,0,0


In [441]:
main_df.columns

Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'ca', 'target', 'cp_1.0', 'cp_2.0', 'cp_3.0', 'cp_4.0', 'restecg_0.0',
       'restecg_1.0', 'restecg_2.0', 'slope_1.0', 'slope_2.0', 'slope_3.0',
       'thal_3.0', 'thal_6.0', 'thal_7.0'],
      dtype='object')

In [442]:
y = main_df["target"].to_numpy().astype(np.float64)
X = main_df.drop(columns="target").to_numpy().astype(np.float64)


train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.6)

## 3.4. Logistic Regression

In [443]:
logistic_regression_model = LogisticRegression(max_iter=1000)

In [444]:
logistic_regression_model.fit(train_X, train_y)

C:\Users\aniru\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [445]:
train_yhat = logistic_regression_model.predict(train_X)

In [446]:
accuracy_score(train_y, train_yhat)

0.8895027624309392

In [447]:
f1_score(train_y, train_yhat)

0.8795180722891566

In [449]:
test_yhat = logistic_regression_model.predict(test_X)

In [451]:
print(accuracy_score(test_y, test_yhat), f1_score(test_y, test_yhat))

0.7592592592592593

Not bad for a first attempt. You can read about the F1 score here. My main motivation for F1 score as opposed to just accuracy is this -- if a dataset is imbalanced and y = 0 in 90% of the cases, then a model that only predicts 0 for any input will have a 90% accuracy! 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html 

https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9 

https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/ 

## 3.5. Naive Bayes (a couple of flavours)

We will be trying two types of Naive Bayes here -- Gaussian, and Bernoulli. The method is pretty much the same, the only difference is that in Gaussian Naive Bayes, we assume that every feature in X is sampled from a Gaussian, while in Bernoulli Naive Bayes, we assume that every feature in X is sampled from a Bernoulli distribution.

### 3.5.1 Gaussian 

In [452]:
gnb = GaussianNB()
gnb.fit(train_X, train_y)

GaussianNB()

In [453]:
train_yhat = gnb.predict(train_X)

In [454]:
accuracy_score(train_y, train_yhat)

0.8176795580110497

In [455]:
f1_score(train_y, train_yhat)

0.7785234899328859

In [456]:
test_yhat = gnb.predict(test_X)
print(accuracy_score(test_y, test_yhat), f1_score(test_y, test_yhat))

0.7704918032786885 0.7142857142857143


### 3.5.2 Bernoulli

In [458]:
bnb = BernoulliNB()
bnb.fit(train_X, train_y)

BernoulliNB()

In [459]:
yhat = bnb.predict(train_X)

In [460]:
accuracy_score(train_y, train_yhat)

0.8176795580110497

In [461]:
f1_score(train_y, train_yhat)

0.7785234899328859

In [462]:
test_yhat = bnb.predict(test_X)
print(accuracy_score(test_y, test_yhat), f1_score(test_y, test_yhat))

0.819672131147541 0.8035714285714286


## 3.6. Advanced topic (optional)

Here, we have a mixture of continuous and categorical data. Therefore, a good option would be: 

1. Label encode the categorical columns. 
2. Perform Gaussian NB using just the continuous columns and MultinomialNB using the label-encoded categorical columns. 
3. Combine these two models to obtain the final prediction as given here: https://medium.com/analytics-vidhya/naive-bayes-for-mixed-typed-data-in-scikit-learn-fb6843e241f0 

We can discuss this further in #doubts or in the TA sessions.